In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rs
import earthpy.plot as ep
import pandas as pd

In [ ]:
jan = rs.open('./img/resampled/Subset_9_jan_2021_resampled.tif')
apr = rs.open('./img/resampled/Subset_19_april_2021_resampled.tif')
mar = rs.open('./img/resampled/Subset_20_march_2021_resampled.tif')

In [ ]:
jan.meta

In [ ]:
pipelatlong = pd.read_csv('pipe_lat_long.txt', sep="\t")
cols = 6261
rows = 2451
top, left, bottom, right = 29.985, 77.504, 29.75, 78.145

pipeindices = []
k = pipelatlong.shape[0]
for i in range(k):
    yc = pipelatlong.loc[i].at["Latitude"]
    xc = pipelatlong.loc[i].at["Longitude"]
    yi = round((yc - top)*(rows - 1)/(bottom - top))
    xi = round((xc - left)*(cols - 1)/(right - left))
    pipeindices.append((yi, xi))

In [ ]:
np.seterr(divide='ignore', invalid='ignore')
def calculate_ndvi(img):
    red = img.read(4).astype(float)
    nir8 = img.read(7).astype(float)
    return ((nir8 - red)/(nir8 + red))

In [ ]:
def plot_ndvi(a_ndvi, month):
    ti = "NDVI plot of "+month
    ep.plot_bands(a_ndvi,
                  cmap="PiYG",
                  vmin=np.nanmin(a_ndvi), vmax=np.nanmax(a_ndvi),
                  scale=False,
                  title=ti,
                  figsize=(30,30))
    plt.show()

In [ ]:
def plot_ndvi_diff(a_ndvi, b_ndvi, month1, month2):
    ti = "Plot of Difference between NDVIs of "+month1+" and "+month2
    k = (a_ndvi - b_ndvi)
    ep.plot_bands(k,
                  vmin=np.nanmin(k), vmax=np.nanmax(k),
                  scale=False,
                  title=ti)
    plt.show()

In [ ]:
def Anamoly(a_ndvi, b_ndvi, threshold, r):
    if (a_ndvi.shape[0] != b_ndvi.shape[0] or a_ndvi.shape[1] != b_ndvi.shape[1]):
        print("Two arrays are not the same size\n")
        return []
    n = a_ndvi.shape[0]
    m = a_ndvi.shape[1]
    list = []
    for (cy, cx) in pipeindices:
        cnt = 0
        value = 0
        for i in range(2*r):
            for j in range(2*r):
                if (cy + i - r >= 0 and cx + j - r >= 0 and cy + i - r <= n and cx + j - r <= m):
                    cnt = cnt + 1
                    value += abs(a_ndvi[cy+i-r][cx+j-r] - b_ndvi[cy+i-r][cx+j-r])
        if (value >= cnt*threshold):
            list.append(1)
        else:
            list.append(0)
    return list    

In [ ]:
from matplotlib.patches import Circle
img = mar.read(7)

def plot(x, img, month1, month2, ndvi):
    ti = "Plot of anomalies based on "+ndvi+" between "+month1+" and "+month2
    patches = [Circle((cy, cx), radius=20, color='yellow') for (cx, cy) in pipeindices]
    patches2 = [Circle((cy, cx), radius=5, color='white') for (cx, cy) in pipeindices]
    patches3 = []
    for i in range(len(x)):
        if (x[i] == 1):
            (cx, cy) = pipeindices[i]
            patches3.append(Circle((cy, cx), radius=20, color='red'))
    fig, ax = plt.subplots(figsize=(20,20), dpi=150)
    ax.imshow(img)
    for p in patches:
        ax.add_patch(p)
    for p in patches3:
        ax.add_patch(p)
    for p in patches2:
        ax.add_patch(p)
    plt.title(ti)
    plt.show(fig)

In [ ]:
jan_ndvi = calculate_ndvi(jan)
mar_ndvi = calculate_ndvi(mar)
apr_ndvi = calculate_ndvi(apr)

In [ ]:
plot_ndvi(jan_ndvi, month="January")

In [ ]:
plot_ndvi(mar_ndvi,month="March")

In [ ]:
plot_ndvi(apr_ndvi, month="April")

In [ ]:
plot_ndvi_diff(jan_ndvi, mar_ndvi, "January", "March")

In [ ]:
plot_ndvi_diff(jan_ndvi, apr_ndvi, "January", "April")

In [ ]:
plot_ndvi_diff(mar_ndvi, apr_ndvi, "March", "April")

In [ ]:
janapr = Anamoly(jan_ndvi, mar_ndvi, 0.25, 10)
plot(janapr, img, "January", "March", "NDVI")

In [ ]:
marapr = Anamoly(mar_ndvi, apr_ndvi, 0.4, 10)
plot(marapr, img, "March", "April", "NDVI")

In [ ]:
janmar = Anamoly(jan_ndvi, apr_ndvi, 0.1, 10)
plot(janapr, img, "January", "April", "NDVI")

In [ ]:
def calculate_ndbi(img):
    swir = img.read(11).astype(float)
    nir = img.read(8).astype(float)
    return ((swir - nir)/(swir+nir))

In [ ]:
def plot_ndbi(a_ndbi, month):
    ti="NDBI plot of month "+month
    ep.plot_bands(a_ndbi,
                  vmin=np.nanmin(a_ndbi), vmax=np.nanmax(a_ndbi),
                  figsize=(15,15),
                  scale=False,
                  title=ti)
    plt.show()

In [ ]:
jan_ndbi = calculate_ndbi(jan)
mar_ndbi = calculate_ndbi(mar)
apr_ndbi = calculate_ndbi(apr)

In [ ]:
plot_ndbi(jan_ndbi, "January")

In [ ]:
plot_ndbi(mar_ndbi, "March")

In [ ]:
plot_ndbi(apr_ndbi, "April")

In [ ]:
janmar_ndbi = Anamoly(jan_ndbi, mar_ndbi, 0.3, 10)
plot(janmar_ndbi, img, "January", "March", "NDBI")

In [ ]:
janapr_ndbi = Anamoly(jan_ndbi, apr_ndbi, 0.3, 10)
plot(janapr_ndbi, img, "January", "April", "NDBI")

In [ ]:
marapr_ndbi = Anamoly(mar_ndbi, apr_ndbi, 0.3, 10)
plot(marapr_ndbi, img, "March", "April", "NDBI")